In [1]:
import pandas as pd 
import numpy as np 
import configparser
import os
import seaborn as sns
import plotly_express as px

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

import nltk 

config = configparser.ConfigParser()
config.read("env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
base_path = config['DEFAULT']['base_path']
code_dir = config['DEFAULT']['code_dir']

data_prefix = 'Maha'
out_path = f'{output_dir}/{data_prefix}'
OHCO = ['book_id','chap_id','sec_id','para_num', 'sent_num', 'token_num']


os.chdir(code_dir)
import preprocess
import bow_analysis
os.chdir(base_path)

In [2]:
CORPUS = pd.read_csv(fr"{output_dir}\F3\{data_prefix}-CORPUS.csv", sep="|").set_index(OHCO)
LIB = pd.read_csv(fr"{output_dir}\F3\{data_prefix}-LIB.csv", sep="|").set_index('book_id')
LIB2 = pd.read_csv(fr"{output_dir}\F3\{data_prefix}-LIB2.csv", sep="|").set_index(OHCO[:2])
VOCAB = pd.read_csv(fr"{output_dir}\CHAP_BOW\{data_prefix}-VOCAB2.csv", sep="|")
TFIDF = pd.read_csv(fr"{output_dir}\CHAP_BOW\{data_prefix}-TFIDF_REDUCED_CHAPS_L2.csv", sep="|").set_index(OHCO[:2])

In [3]:
PARA = OHCO[:4]
SECS = OHCO[:3]
CHAP = OHCO[:2]
BOOK = OHCO[:1]

In [4]:
BAG = CHAP

In [5]:
ngram_range = (1, 2)
n_terms = 1008
n_topics = 20
max_iter = 100
n_top_terms = 7
colors = "YlGnBu"

In [6]:
DOCS = CORPUS[CORPUS.pos.str.match(r'^NNS?$')]\
    .groupby(BAG).term_str\
    .apply(lambda x: ' '.join(map(str,x)))\
    .to_frame()\
    .rename(columns={'term_str':'doc_str'})
DOCS.head()

doc_str
book_id chap_id                                                   
1       1        male goddess word son humility day sages vows ...
        2        son account place descriptions ye men deserve ...
        3        son brothers sacrifice plains brothers sacrifi...
        4        son forest twelve years sacrifice attendance d...
        5        reason tiger kings royal determine lives snake...

In [7]:
from sklearn.feature_extraction import text
custom_stopwords = list(text.ENGLISH_STOP_WORDS.union([
    'thou', 'thee', 'hath', 'thy', 'art', 'ye', 'hast',
    'king', 'son', 'sons', 'men',  
]))

In [8]:
count_engine = CountVectorizer(max_features=n_terms, ngram_range=ngram_range, stop_words=custom_stopwords)
count_model = count_engine.fit_transform(DOCS.doc_str)
TERMS = count_engine.get_feature_names_out()

In [9]:
VOCAB = pd.DataFrame(index=TERMS)
VOCAB.index.name = 'term_str'
DTM = pd.DataFrame(count_model.toarray(), index=DOCS.index, columns=TERMS)
DTM

ablutions  abode  absence  abstention  accomplishment  \
book_id chap_id                                                          
1       1                0      5        0           0               1   
        2                0      3        0           0               0   
        3                2      3        1           0               0   
        4                1      2        0           0               0   
        5                3     10        0           0               3   
...                    ...    ...      ...         ...             ...   
15      2                0      3        0           0               2   
        3                1      1        0           1               1   
16      1                0      2        0           0               0   
17      1                0      0        0           0               1   
18      1                0      5        0           0               1   

                 accomplishments  account  achievements  acquisition  act  \
book_id chap_id                                                             
1       1                      0        2             0            1    1   
        2                      0        4             0            2    1   
        3                      0        1             0            0    2   
        4                      0        2             0            0    1   
        5                      0        8             4            2   19   
...                          ...      ...           ...          ...  ...   
15      2                      0        5             1            0    1   
        3                      0        0             0            0    0   
16      1                      0        2             0            0    1   
17      1                      0        0             1            0    3   
18      1                      0        1             1            1    4   

                 ...  worship  worthy  wrath  wretch  year  years  yoga  yore  \
book_id chap_id  ...                                                            
1       1        ...        0       1      0       1     0      2     0     0   
        2        ...        0       0      8       1     1      0     1     0   
        3        ...        0       0      1       2     5      0     0     0   
        4        ...        0       0      3       0     0      2     0     2   
        5        ...        0       1     10       9     1     11     0     2   
...              ...      ...     ...    ...     ...   ...    ...   ...   ...   
15      2        ...        1       0      3       0     1      4     0     0   
        3        ...        0       0      0       0     0      3     0     0   
16      1        ...        0       0      5       1     3      0     1     0   
17      1        ...        0       0      0       0     0      0     0     0   
18      1        ...        1       0      4       0     0      5     0     0   

                 youth  yudhishthira  
book_id chap_id                       
1       1            0             2  
        2            1             2  
        3            0             0  
        4            0             0  
        5            0             0  
...                ...           ...  
15      2            0             0  
        3            0             1  
16      1            0             0  
17      1            0             2  
18      1            0             2  

[99 rows x 1008 columns]

In [10]:
VOCAB['doc_count'] = DTM.astype('bool').astype('int').sum()
DOCS['term_count'] = DTM.sum(1)
VOCAB

,doc_count
term_str,
ablutions,24
abode,86
absence,27
abstention,20
accomplishment,55
...,...
years,81
yoga,24
yore,44


In [11]:
DOCS.term_count.describe()

count       99.000000
mean      3808.525253
std       6709.625842
min         41.000000
25%        863.000000
50%       1430.000000
75%       3359.000000
max      40880.000000
Name: term_count, dtype: float64

In [12]:
lda_engine = LDA(n_components=n_topics, max_iter=max_iter, learning_offset=50., random_state=0)

In [13]:
TNAMES = [f"T{str(x).zfill(len(str(n_topics)))}" for x in range(n_topics)]
TNAMES

['T00',
 'T01',
 'T02',
 'T03',
 'T04',
 'T05',
 'T06',
 'T07',
 'T08',
 'T09',
 'T10',
 'T11',
 'T12',
 'T13',
 'T14',
 'T15',
 'T16',
 'T17',
 'T18',
 'T19']

In [14]:
lda_model = lda_engine.fit_transform(count_model)

In [15]:
THETA = pd.DataFrame(lda_model, index=DOCS.index)
THETA.columns.name = 'topic_id'
THETA.columns = TNAMES

In [16]:
THETA.sample(10).T.style.background_gradient(cmap=colors, axis=None)

In [17]:
PHI = pd.DataFrame(lda_engine.components_, columns=TERMS, index=TNAMES)
PHI.index.name = 'topic_id'
PHI.columns.name = 'term_str'

PHI.T.sample(10).style.background_gradient(cmap=colors, axis=None)

topic_id,T00,T01,T02,T03,T04,T05,T06,T07,T08,T09,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19
term_str,,,,,,,,,,,,,,,,,,,,
beings,0.050000,0.050000,33.391908,19.450234,0.050000,40.135115,0.050000,0.050000,41.456816,59.095220,0.050000,117.665335,0.050000,271.341647,162.170761,9.713761,31.790697,0.050000,8.338507,0.050000
contact,1.621165,0.050000,0.050000,0.050000,0.050000,4.957101,6.508057,0.050000,14.150194,17.785595,0.050000,0.050000,0.050000,39.191097,7.136790,0.050000,0.050000,0.050000,0.050000,0.050000
emancipation,0.050000,0.050000,0.050000,0.050000,10.657519,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,175.442481,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
affection,77.419492,0.050000,0.050000,0.050000,75.562955,72.458902,19.666143,0.050000,0.050000,18.483254,0.050000,0.050000,0.050000,65.429992,37.011206,8.661155,0.050000,11.504652,2.979031,28.373218
backs,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,71.418032,0.050000,13.059235,0.050000,0.050000,2.558631,0.050000,0.050000,0.050000,0.050000,9.164101
tirthas,0.050000,0.050000,0.050000,0.050000,3.761639,2.240063,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,6.068466,0.050000,128.129832,0.050000,0.050000,0.050000
qualities,0.050000,0.050000,0.050000,0.050000,40.548017,0.050000,0.050000,0.050000,36.865096,0.050000,0.050000,0.050000,0.050000,86.542981,0.050000,110.537836,0.050000,0.050000,0.050000,19.756071
ashes,0.050000,0.050000,0.050000,0.050000,7.579909,14.810770,4.361845,3.322224,0.050000,8.346233,0.050000,13.243562,0.050000,3.082027,27.431570,0.050000,6.765182,11.918218,1.688459,0.050000
search,0.050000,0.050000,0.050000,0.050000,8.525556,25.411656,0.050000,0.050000,0.050000,0.050000,3.129511,0.050000,0.050000,7.672474,5.857893,0.050000,6.159047,31.544981,1.098883,0.050000


In [18]:
TOPICS = PHI.stack().groupby('topic_id')\
    .apply(lambda x: ' '.join(x.sort_values(ascending=False).head(n_top_terms).reset_index().term_str))\
    .to_frame('top_terms')

In [19]:
TOPICS.style

,top_terms
topic_id,
T00,words race kings peace earth battle virtue
T01,attachments swans creature possessions steps contact contentment
T02,deities gods god goddess worlds beings form
T03,sacrifice sacrifices earth kings gods monarch history
T04,person life duties wealth persons man world
T05,daughter wife father monarch time race earth
T06,words race foremost grief monarch penances kings
T07,monkeys death mother forest children words house
T08,gods world virtue man earth lord food


In [20]:
TOPICS['doc_weight_sum'] = THETA.sum()
TOPICS['term_freq'] = PHI.sum(1) / PHI.sum(1).sum()
TOPICS.sort_values('doc_weight_sum', ascending=False).style.background_gradient(cmap=colors)

,top_terms,doc_weight_sum,term_freq
topic_id,,,
T09,battle arrows shafts car steeds warriors elephants,14.769579,0.240429
T05,daughter wife father monarch time race earth,11.777855,0.076945
T19,kings race monarch wealth foes bull foremost,10.249686,0.037980
T17,city mighty lord words arms car success,8.404426,0.039021
T11,gods energy creatures weapons lord foremost weapon,7.766597,0.060173
T13,acts knowledge creatures mind senses body viz,6.913124,0.151486
T00,words race kings peace earth battle virtue,5.421011,0.038928
T08,gods world virtue man earth lord food,4.845075,0.029069
T04,person life duties wealth persons man world,4.777370,0.081840


In [21]:
LIB2

chap_name              book_name  \
book_id chap_id                                                 
1       1           Anukramanika Parva              Adi Parva   
        2               Sangraha Parva              Adi Parva   
        3                Paushya Parva              Adi Parva   
        4                Pauloma Parva              Adi Parva   
        5                 Astika Parva              Adi Parva   
...                                ...                    ...   
15      2           Putradarsana Parva     Asramavasika Parva   
        3           Naradagamana Parva     Asramavasika Parva   
16      1                Mausala Parva          Mausala Parva   
17      1        Mahaprasthanika Parva  Mahaprasthanika Parva   
18      1         Svargarohanika Parva   Svargarohanika Parva   

                source_file_path  chap_len  n_secs  n_sents  n_tokens  
book_id chap_id                                                        
1       1             maha01.txt      7511       1      374      7511  
        2             maha01.txt      8655       1      524      8655  
        3             maha01.txt      6137       1      396      6137  
        4             maha01.txt      4661       9      261      4661  
        5             maha01.txt     30619      46     1753     30619  
...                          ...       ...     ...      ...       ...  
15      2             maha15.txt      6789       8      417      6789  
        3             maha15.txt      2551       3      149      2551  
16      1             maha16.txt      8072       7      432      8072  
17      1             maha17.txt      3062       2      206      3062  
18      1             maha18.txt      8628       5      495      8628  

[99 rows x 7 columns]

In [22]:
THETA2 = THETA.join(LIB2)
THETA2

T00       T01       T02       T03       T04       T05  \
book_id chap_id                                                               
1       1        0.000054  0.000054  0.000054  0.136817  0.000054  0.000054   
        2        0.000048  0.000048  0.000048  0.125372  0.016411  0.215567   
        3        0.000068  0.000068  0.000068  0.000068  0.000068  0.445287   
        4        0.000091  0.000091  0.000091  0.069001  0.000091  0.469267   
        5        0.000012  0.000012  0.000012  0.104952  0.000012  0.413447   
...                   ...       ...       ...       ...       ...       ...   
15      2        0.000046  0.000046  0.025514  0.000046  0.000046  0.032971   
        3        0.000134  0.000134  0.000134  0.000134  0.000134  0.000134   
16      1        0.000043  0.000043  0.008785  0.000043  0.050319  0.000043   
17      1        0.000113  0.000113  0.041347  0.000113  0.000113  0.000113   
18      1        0.000038  0.000038  0.059535  0.050083  0.000038  0.000038   

                      T06       T07       T08       T09  ...       T17  \
book_id chap_id                                          ...             
1       1        0.000054  0.000054  0.218546  0.000054  ...  0.492993   
        2        0.000048  0.000048  0.098142  0.031687  ...  0.218050   
        3        0.000068  0.058074  0.000068  0.000068  ...  0.000068   
        4        0.000091  0.000091  0.076657  0.000091  ...  0.000091   
        5        0.000012  0.000012  0.000012  0.000012  ...  0.000012   
...                   ...       ...       ...       ...  ...       ...   
15      2        0.600137  0.000046  0.000046  0.025335  ...  0.000046   
        3        0.616071  0.243535  0.000134  0.000134  ...  0.000134   
16      1        0.316486  0.008276  0.000043  0.088782  ...  0.050394   
17      1        0.515413  0.000113  0.000113  0.000113  ...  0.052797   
18      1        0.203317  0.000038  0.000038  0.035605  ...  0.000038   

                      T18       T19              chap_name  \
book_id chap_id                                              
1       1        0.000054  0.000054     Anukramanika Parva   
        2        0.000048  0.000048         Sangraha Parva   
        3        0.000068  0.000068          Paushya Parva   
        4        0.000091  0.000091          Pauloma Parva   
        5        0.000012  0.000012           Astika Parva   
...                   ...       ...                    ...   
15      2        0.000046  0.000046     Putradarsana Parva   
        3        0.000134  0.000134     Naradagamana Parva   
16      1        0.230120  0.025940          Mausala Parva   
17      1        0.000113  0.104977  Mahaprasthanika Parva   
18      1        0.053005  0.006546   Svargarohanika Parva   

                             book_name  source_file_path  chap_len  n_secs  \
book_id chap_id                                                              
1       1                    Adi Parva        maha01.txt      7511       1   
        2                    Adi Parva        maha01.txt      8655       1   
        3                    Adi Parva        maha01.txt      6137       1   
        4                    Adi Parva        maha01.txt      4661       9   
        5                    Adi Parva        maha01.txt     30619      46   
...                                ...               ...       ...     ...   
15      2           Asramavasika Parva        maha15.txt      6789       8   
        3           Asramavasika Parva        maha15.txt      2551       3   
16      1                Mausala Parva        maha16.txt      8072       7   
17      1        Mahaprasthanika Parva        maha17.txt      3062       2   
18      1         Svargarohanika Parva        maha18.txt      8628       5   

                 n_sents  n_tokens  
book_id chap_id                     
1       1            374      7511  
        2            524      8655  
        3            396      6137  
        4            261

In [23]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
DCM = pd.DataFrame(pca.fit_transform(THETA), index=THETA.index)
DCM.columns = ['PC{}'.format(i) for i in DCM.columns]
DCM = DCM.join(LIB2[['book_name','chap_name']], on=OHCO[:2])
DCM


PC0       PC1       PC2       PC3       PC4       PC5  \
book_id chap_id                                                               
1       1       -0.157221 -0.176634  0.009030  0.137213  0.382744 -0.054882   
        2       -0.131843 -0.000274  0.128133  0.025682  0.125139 -0.045956   
        3       -0.197393  0.093185  0.300510 -0.188778 -0.052398  0.090845   
        4       -0.204317  0.141186  0.309996 -0.144714 -0.028152  0.017392   
        5       -0.181243  0.120444  0.279516  0.053856 -0.149001  0.118972   
...                   ...       ...       ...       ...       ...       ...   
15      2       -0.112244 -0.273625 -0.062753 -0.187363 -0.267452 -0.209405   
        3       -0.121257 -0.222583 -0.099279 -0.089269 -0.334599 -0.356027   
16      1       -0.022928 -0.161052 -0.068763  0.046644 -0.216249 -0.199974   
17      1       -0.130558 -0.158131 -0.157640 -0.051280 -0.272961 -0.255627   
18      1       -0.080411 -0.234114 -0.088032 -0.115105 -0.226609 -0.047352   

                      PC6       PC7       PC8       PC9  \
book_id chap_id                                           
1       1       -0.159197 -0.001489 -0.036458 -0.002017   
        2       -0.040852  0.097339  0.111514 -0.020800   
        3       -0.030700  0.046561 -0.025956 -0.026813   
        4        0.005454  0.042653  0.029778  0.082471   
        5        0.040170 -0.030273  0.016953  0.043888   
...                   ...       ...       ...       ...   
15      2       -0.294420 -0.105296  0.026370  0.002776   
        3       -0.276720 -0.143349  0.056718  0.007709   
16      1       -0.082039  0.079223 -0.085360  0.056951   
17      1       -0.266409 -0.128589  0.039255 -0.002879   
18      1       -0.182799 -0.035460 -0.036128 -0.063331   

                             book_name              chap_name  
book_id chap_id                                                
1       1                    Adi Parva     Anukramanika Parva  
        2                    Adi Parva         Sangraha Parva  
        3                    Adi Parva          Paushya Parva  
        4                    Adi Parva          Pauloma Parva  
        5                    Adi Parva           Astika Parva  
...                                ...                    ...  
15      2           Asramavasika Parva     Putradarsana Parva  
        3           Asramavasika Parva     Naradagamana Parva  
16      1                Mausala Parva          Mausala Parva  
17      1        Mahaprasthanika Parva  Mahaprasthanika Parva  
18      1         Svargarohanika Parva   Svargarohanika Parva  

[99 rows x 12 columns]

In [24]:
LOADINGS = pd.DataFrame(pca.components_.T * np.sqrt(pca.explained_variance_))
LOADINGS.index = THETA.columns
LOADINGS.columns = ["PC{}".format(i) for i in LOADINGS.columns]
LOADINGS.index.name = 'topic_id'

LOADINGS = LOADINGS.join(TOPICS)
LOADINGS.T

topic_id,T00,T01,T02,T03,T04,T05,T06,T07,T08,T09,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19
PC0,-0.004854,-0.000027,-0.002128,-0.015115,-0.017895,-0.067703,-0.018004,-0.008002,-0.021669,0.268274,0.006924,0.000263,-0.00689,-0.023096,-0.01094,-0.004146,-0.007743,-0.029409,-0.002215,-0.035627
PC1,-0.002439,0.000006,-0.00451,0.009307,-0.024525,0.10835,-0.037579,0.002168,-0.040379,0.027925,-0.008606,-0.012976,-0.000284,-0.077437,-0.029266,-0.002302,-0.006896,0.002811,-0.017806,0.114439
PC2,-0.010528,-0.000002,-0.003374,-0.025392,-0.00681,0.115802,-0.017015,0.000717,0.011615,0.013899,-0.003718,0.017925,0.009165,0.006587,-0.012313,0.007363,0.001319,0.012476,-0.005567,-0.112148
PC3,0.01194,0.000001,-0.001511,-0.002203,-0.028277,-0.035532,-0.021456,0.005302,-0.010634,-0.014254,0.009787,0.100505,-0.003389,-0.066828,-0.019753,-0.008784,0.014201,0.066317,0.020017,-0.015448
PC4,0.024503,0.000025,-0.003582,0.005764,0.005144,-0.013073,-0.056094,-0.011273,0.053793,0.008502,-0.007157,-0.049309,0.001964,0.023652,-0.034988,-0.002638,-0.007726,0.084605,-0.02816,0.006047
PC5,-0.009423,-0.000005,0.001475,0.010113,-0.040756,0.001795,-0.065899,-0.012462,-0.005757,-0.000608,-0.001576,0.055378,-0.001018,0.076837,0.003266,0.007039,0.006361,-0.03112,-0.016643,0.023002
PC6,0.019936,-0.000021,-0.004214,0.004624,0.090835,-0.003394,-0.052337,-0.000271,0.008898,-0.00518,0.007126,0.017476,0.001317,-0.019052,-0.026864,-0.00402,0.002394,-0.043237,0.010658,-0.004673
PC7,0.001442,0.000006,-0.001516,-0.002086,-0.026294,0.000409,-0.026817,-0.010365,-0.017281,-0.007431,0.064012,-0.03423,0.005515,0.005693,-0.01405,0.001455,-0.009393,-0.002285,0.078975,-0.005762
PC8,0.003378,0.000007,0.001179,0.007778,-0.016136,-0.001957,0.008702,-0.002657,0.029772,-0.005369,0.077737,0.000912,0.00079,-0.01969,-0.006733,-0.002932,0.010658,-0.022271,-0.058677,-0.00449
PC9,0.002413,-0.000002,-0.000428,0.023432,-0.033355,0.002057,0.008559,-0.010435,0.071646,0.00091,-0.032385,0.002859,0.002512,-0.019655,-0.022602,-0.004014,0.009576,-0.03513,0.031818,0.002224


In [25]:
def vis_loadings(LOADINGS, a=0, b=1, hover_name='topic_num'):
    return px.scatter(LOADINGS.reset_index(), f"PC{a}", f"PC{b}", 
                      text='topic_id', hover_name='top_terms',
                      size = "doc_weight_sum",
                    #    color='max_pos_group', 
                      marginal_x='box', height=800)


vis_loadings(LOADINGS, 0, 1)
# LOADINGS